In [ ]:
#載入套件
import re
import requests
from bs4 import BeautifulSoup as bs
import os

In [ ]:
import sys
print(sys.version)

3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]


In [ ]:
!pip list | grep -E "^requests |^beautifulsoup4 "

beautifulsoup4                     4.12.3
requests                           2.32.3


In [ ]:
# gutenberg https://www.gutenberg.org/
# use selector : li.pgdbetext > a[href] can find
# url = https://www.gutenberg.org/browse/languages/zh

url = 'https://www.gutenberg.org/browse/languages/zh'
res = requests.get(url)
books_html = res.text
# print(books_html[:500])
pattern = r'<a href="\/ebooks\/(\d+)">(.*?)</a>'
books=re.findall(pattern,books_html)
print(books)

[('25328', '豆棚閒話'), ('24225', '戲中戲'), ('24185', '比目魚'), ('27119', '比目魚'), ('38585', 'Study of Inner Cultivation'), ('12479', '三字經'), ('25146', '山水情'), ('25288', '山海經'), ('23825', '施公案'), ('25393', '施公案'), ('25501', '易經'), ('23938', '木蘭奇女傳'), ('54494', '海公案'), ('24068', '燕丹子'), ('27686', '狄公案'), ('25196', '百家姓'), ('24048', '禮記'), ('27330', '綠牡丹'), ('23873', '詩經'), ('27399', '麟兒報'), ('38585', 'Study of Inner Cultivation'), ('27059', 'Hu Die Mei\r<br>Yuan Yang Meng'), ('27060', 'Qing Lou Meng<br>Qi Hong Xiao Shi'), ('26904', '天豹圖'), ('26886', '梁公九諫'), ('25352', '長恨歌'), ('24051', '李娃傳'), ('25422', '玉樓春'), ('23841', '漢書'), ('26921', '引鳳蕭'), ('24230', '今古奇觀'), ('27332', '後西游記'), ('27331', '飛跎全傳'), ('23585', '佛說四十二章經'), ('24264', '紅樓夢'), ('24041', '洛神賦'), ('43014', '晁氏儒言 一卷'), ('25217', '水滸後傳'), ('52269', '幼學瓊林'), ('26932', '治世餘聞'), ('25246', '琵琶記'), ('26739', '雪月梅傳'), ('26873', '龍川詞'), ('25606', '三國志'), ('23835', '隋唐演義'), ('23839', '論語'), ('62791', '滬語開路 = Conversational Exercises in the Sha

In [ ]:
# 定義清理書名的凾式

# 可能有以下這些老六
# <a href="/ebooks/12479">三字經</a>
# <a href="/ebooks/7376">中庸 章句</a>
# <a href="/ebooks/7271">搜神記 volume 11-14</a>
# <a href="/ebooks/38585">Study of Inner Cultivation</a>
# 滬語開路 = Conversational Exercises in the Shanghai Dialect
# 兵法 (Bīng Fǎ)
# 中文 : \u4e00-\u9fff
# 留下中文與特定符號，刪去沒有中文的
def clean_title(book_title):
    pattern = r'[^\u4e00-\u9fff0-9\(\)\-.,]'
    cleaned_title = re.sub(pattern,'',book_title)
    if re.search(r'[\u4e00-\u9fff]',cleaned_title):
        return cleaned_title.strip()
    else:
        return ''
# print(clean_title('兵法 (Bīng Fǎ)'))

In [ ]:
# 建立字典
# ('23825', '施公案'), ('25393', '施公案')可能有一模一樣的書，存檔時再過濾
# booknum as key
book_dict = {}
count = 0
for book in books:
    name = clean_title(book[1])
    if name != '':
        book_dict[book[0]] = name
    if name == '':
        count += 1
        print(book[1])
print(book_dict)
print(len(book_dict))
print(count)

Study of Inner Cultivation
Study of Inner Cultivation
<br>Yuan Yang Meng
Qing Lou Meng<br>Qi Hong Xiao Shi
Eight dwelling places of Buddhist immortals
Dao De Jing: A Minimalist Translation
Dao De Jing: A Minimalist Translation
The Chinese Classics — Volume 1: Confucian Analects
<br>(Shih ching. English) — Volume 1
The Chinese Classics — Volume 1: Confucian Analects
<br>(Shih ching. English) — Volume 1
Dao De Jing: A Minimalist Translation
Study of Inner Cultivation
True Heart/Mind
Dao De Jing: A Minimalist Translation
Study of Inner Cultivation
True Heart/Mind
Wu De Ji
Brief Account of the English Character
Brief Account of the English Character
Brief Account of the English Character
Brief Account of the English Character
<br>Yuan Yang Meng
True Heart/Mind
True Heart/Mind
Peach Blossom Shangri-la: Tao Hua Yuan Ji
Peach Blossom Shangri-la: Tao Hua Yuan Ji
Three Hundred Tang Poems, Volume 1
Bizarre Happenings Eyewitnessed Over Two Decades
Qing Lou Meng<br>Qi Hong Xiao Shi
{'25328': '豆棚閒話

In [ ]:
# 定義清理文本內容的凾式
# 留下有包含中文、符號、數字以及空白的行
# 刪除沒有文字的行
def clean_text(book_text):
    pattern = r'^\u4e00-\u9fff0-9\(\)\-.,~\s'
    cleaned_text = re.sub(pattern,'',book_text)
    if re.search(r'[\u4e00-\u9fff0-9]',cleaned_text):
        return cleaned_text.strip()
    else:
        return ''

In [ ]:
# book url sample https://www.gutenberg.org/cache/epub/12479/pg12479-images.html
# 先用一本測試
# 7376 7271 23825
book_num = '23825'
url = 'https://www.gutenberg.org/cache/epub/'+ book_num +'/pg' + book_num + '-images.html'
# print(url)
# 請求html文字檔
res = requests.get(url)
book_html = res.text
# 用bs4的selector #選擇p標籤內有id元素的
soup = bs(book_html,'lxml')
line=[]
for p in soup.select('p')[:20]:
    if p.has_attr('id'):
      if clean_text(p.text) != '':
        # print(clean_text(p.text))
        line.append(clean_text(p.text))

# filepath = foldername + '/' + book_dict[book_num] + '.txt'
# with open(filepath,'w',encoding='utf-8') as f:
#     f.write('\n'.join(line))

In [ ]:
# 建立資料夾並檢查是否已存在
foldername = 'gutenberg'
if not os.path.exists(foldername):
    os.mkdir(foldername)

In [ ]:
# 將測試結果依照bookdict內的書號跌代執行
for book_num in book_dict:
    url = 'https://www.gutenberg.org/cache/epub/'+ book_num +'/pg' + book_num + '-images.html'
    res = requests.get(url)
    book_html = res.text
    soup = bs(book_html,'lxml')

    line = []
    for p in soup.select('p'):
        if p.has_attr('id'):
          if clean_text(p.text) != '':
            line.append(clean_text(p.text))
    filepath = foldername + '/' + book_dict[book_num] + '.txt'
    with open(filepath,'w',encoding='utf-8') as f:
        f.write('\n'.join(line))


In [ ]:
# 檢查目錄下的檔案數量
len(os.listdir(foldername))

401